# USA Housing Model Notebook 
#### In this notebook we will create, train , validate, and do prediction by using InterSystems SQL Cloud

<h1>Table of contents</h1>
​
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#about_dataset">About dataset</a></li>
        <li><a href="#create">Create table and import data</a></li>
        <li><a href="#cm">Create Model</a></li>
        <li><a href="#tm">Train Model</a></li>
        <li><a href="#vm">Validate Model</a></li>
        <li><a href="#pr">Do Prediction</a></li>   
        <li><a href="#sm">Summary</a></li>  
    </ol>
</div>
<br>
<hr>

In [2]:
#import requried libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

<div id="about_dataset">
    <h1>About dataset</h1>   
</div>

[USA Housing dataset](https://www.kaggle.com/datasets/vedavyasv/usa-housing) is taken from [Kaggle](https://www.kaggle.com/)<br> [LICENCE:Public Domain](https://docs.data.world/en/59261-59714-2--Common-license-types-for-datasets.html)

USA_Housing dataset contains the following columns:

- 'Avg. Area Income': Avg. The income of residents of the city house is located in.
- 'Avg. Area House Age': Avg Age of Houses in the same city
- 'Avg. Area Number of Rooms': Avg Number of Rooms for Houses in the same city
- 'Avg. Area Number of Bedrooms': Avg Number of Bedrooms for Houses in the same city
- 'Area Population': The population of city house is located in
- 'Price': Price that the house sold at

In [3]:
#use pandas read_csv function to read CSV file
USAhousing = pd.read_csv('./datasets/USA_Housing_train.csv')
#View first 5 records of dataset
USAhousing.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [4]:
#View information of dataset
USAhousing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Avg. Area Income              4000 non-null   float64
 1   Avg. Area House Age           4000 non-null   float64
 2   Avg. Area Number of Rooms     4000 non-null   float64
 3   Avg. Area Number of Bedrooms  4000 non-null   float64
 4   Area Population               4000 non-null   float64
 5   Price                         4000 non-null   float64
dtypes: float64(6)
memory usage: 187.6 KB


In [5]:
#Descriptive statistics include those that summarize the central tendency, 
#dispersion and shape of a dataset’s distribution
USAhousing.describe()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4.000000e+03
mean,68561.764989,5.982444,6.989255,3.974775,36279.220348,1.235349e+06
std,10693.493261,0.990669,1.004391,1.231435,9899.991119,3.544019e+05
min,17796.631190,2.644304,3.236194,2.000000,172.610686,1.593866e+04
25%,61474.294678,5.334776,6.308615,3.140000,29550.400770,9.997036e+05
50%,68655.504150,5.969464,7.001368,4.045000,36240.428580,1.237303e+06
75%,75781.049625,6.663936,7.665612,4.490000,43000.807275,1.469933e+06
max,107701.748400,9.519088,10.759588,6.500000,69592.040240,2.469066e+06


In [6]:
#create traning table
table = "usa_housing_train"
try:
    #execute the cursor
    cur.execute(f"DROP TABLE {table}")
    cur.execute(f"CREATE TABLE {table} ( id INT(32) AUTO_INCREMENT PRIMARY KEY, AvgAreaIncome FLOAT, AvgAreaHouseAge FLOAT,AvgAreaRooms FLOAT,AvgAreaBedRooms FLOAT,AreaPopulation FLOAT,Price FLOAT)")
except Exception as e:
    print(e)    

#create validate table    
table = "usa_housing_validate"    
try:
    #execute the cursor
    cur.execute(f"DROP TABLE {table}")
    cur.execute(f"CREATE TABLE {table} ( id INT(32) AUTO_INCREMENT PRIMARY KEY, AvgAreaIncome FLOAT, AvgAreaHouseAge FLOAT,AvgAreaRooms FLOAT,AvgAreaBedRooms FLOAT,AreaPopulation FLOAT,Price FLOAT)")
except Exception as e:
    print(e)        

name 'cur' is not defined
name 'cur' is not defined


<div id="create">
    <h1>Create table and import data</h1>   
</div>

In [7]:
#Create connection with InterSystems Cloud SQL
import  utility
connection = utility.get_db_connection()
#open cursor
try:
    cur = connection.cursor()
except:
    print("Connection Cursor Error")
    
print("Connected to IS SQL Cloud")    

Connected to IS SQL Cloud


## Import data

In [8]:
#Read train data
USAhousingTrain = pd.read_csv('./datasets/USA_Housing_train.csv')
#Truncate table and insert records into the table
#PLEASE NOTE THAT BASED ON THE DATA, IT MIGHT TOOK SOME TIME
table = "usa_housing_train"  
try:
    cur.execute(f"TRUNCATE TABLE {table}")
    for i in range(len(USAhousingTrain)):
        cur.execute(f"INSERT INTO {table} (AvgAreaIncome, AvgAreaHouseAge, AvgAreaRooms, AvgAreaBedRooms, AreaPopulation, Price) VALUES ({USAhousingTrain.iloc[i, 0]}, {USAhousingTrain.iloc[i, 1]}, {USAhousingTrain.iloc[i, 2]} , {USAhousingTrain.iloc[i, 3]}, {USAhousingTrain.iloc[i, 4]}, {USAhousingTrain.iloc[i, 5]})")
    #commit changes
    connection.commit()
except Exception as e:
    print(e)      


In [ ]:
#Read validate data
USAhousingVal = pd.read_csv('./datasets/USA_Housing_validate.csv')
#Truncate table and insert records into the table
table = "usa_housing_validate"  
try:
    cur.execute(f"TRUNCATE TABLE {table}")
    for i in range(len(USAhousingVal)):
        cur.execute(f"INSERT INTO {table} (AvgAreaIncome, AvgAreaHouseAge, AvgAreaRooms, AvgAreaBedRooms, AreaPopulation, Price) VALUES ({USAhousingTrain.iloc[i, 0]}, {USAhousingTrain.iloc[i, 1]}, {USAhousingTrain.iloc[i, 2]} , {USAhousingTrain.iloc[i, 3]}, {USAhousingTrain.iloc[i, 4]}, {USAhousingTrain.iloc[i, 5]})")
    #commit changes
    connection.commit()
except Exception as e:
    print(e)      
       

<div id="cm">
    <h1>Create Model</h1>   
</div>

In [ ]:
#Create model by specifying model name, prediction column and train table
modelName = "USAHousingPriceModel"
dataColumn = "Price"
dataTable= "usa_housing_train"
#Create model by using cursor
try:
    cur.execute(f"CREATE MODEL {modelName} PREDICTING ({dataColumn})  FROM {dataTable}") 
except Exception as e:
    print(e)         

connection.commit()
print('Model created')

<div id="tm">
    <h1>Train Model</h1>   
</div>

In [ ]:
# Train model - PLease note it might take time based on number of records
modelName = "USAHousingPriceModel"
dataTable= "usa_housing_train"
try:
    cur.execute(f"TRAIN MODEL {modelName} FROM {dataTable}")
except Exception as e:
    print(e)  
    
connection.commit()
print('Model Trained')

<div id="vm">
    <h1>Validate Model</h1>   
</div>

In [ ]:
#Validate Model 
modelName = "USAHousingPriceModel"
ValidationRunName = "USAHousingPriceValidateuse"
TrainModelValidate="USAHousingPriceModel_t1"
TableToValidate = "usa_housing_validate"
try:
    cur.execute(f"Validate model {modelName} As {ValidationRunName} use {TrainModelValidate} from {TableToValidate}")
except Exception as e:
    print(e)  
    
connection.commit()
print('Model Validated')

#### Printing Validation Metrics

In [ ]:
import utility
modelName = "USAHousingPriceModel"
ValidationRunName = "USAHousingPriceValidateuse"
#get metrics values
mse,rmse,var,r2 = utility.get_validation_metrics(modelName,ValidationRunName)
print("Validation Metrics for Regression Models")
print("MSE = "+ mse)
print("RMSE = "+ rmse)
print("Variance = "+ var)
print("R2 = "+ r2)
cur.close()
connection.close()

When training the model, the accuracy is considered satisfactory if the R2 value is greater than 95%.

<div id="pr">
    <h1>Do Prediction</h1>   
</div>

In [ ]:
import  utility
import pandas as pd
#Create connection
connection = utility.get_db_connection()
#create cursot
cur = connection.cursor()
stat = "SELECT TOP(100) PREDICT(USAHousingPriceModel use USAHousingPriceModel_t1) as prediction, Price, * FROM SQLUser.usa_housing_validate"
#excute statement
cur.execute(stat)
#Fetch all records
data = cur.fetchall()
#create dataframe based on the fetch records
df = pd.DataFrame (data = data, columns = utility.get_cols(cur.description))
#display first 5 records
print(df.head())
cur.close()
connection.close()

<div id="sm">
    <h1>Summary</h1>   
</div>

In this notebook we covered the following:
- Explore USA Housing dataset
- Created table by using IS Cloud Services and imported the data
- Created Model
- Trained Model
- Validate Model
- Done the Pridiction

Thanks